# pySCENIC on NK cells, MM

Preparation and run Single-Cell rEgulatory Network Inference and Clustering using pySCENIC.

## Why pySCENIC
- A lot of compatibility problems with the R version
- Faster
- I like more python than

After having preparred the environment and the files (look [here](./prepare_env.ipnb))

In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
from MulticoreTSNE import MulticoreTSNE as TSNE
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [2]:
adata = sc.read_h5ad("Data/mm_nk.h5ad")
adata

AnnData object with n_obs × n_vars = 14103 × 48361
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent_mt', 'batch', 'label', 'new_label', 'condition'

write to an unfiltered loom file

In [3]:
f_loom_path_unfilt = "Data/nk_mm_unfiltered.loom"
row_attrs = { 
    "Gene": np.array(adata.var.index) ,
}
col_attrs = { 
    "CellID":  np.array(adata.obs.index) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
    "new_label": np.array(adata.obs.new_label)
}

lp.create( f_loom_path_unfilt, adata.X.transpose(), row_attrs, col_attrs )

# basic metrics and filtering:

In [4]:
nCountsPerGene = np.sum(adata.X, axis=0)
nCellsPerGene = np.sum(adata.X>0, axis=0)

# Show info
print("Number of counts (in the dataset units) per gene:", nCountsPerGene.min(), " - " ,nCountsPerGene.max())
print("Number of cells in which each gene is detected:", nCellsPerGene.min(), " - " ,nCellsPerGene.max())

Number of counts (in the dataset units) per gene: 0.0  -  3303925.0
Number of cells in which each gene is detected: 0  -  14081


In [5]:
nCells=adata.X.shape[0]

# pySCENIC thresholds
minCountsPerGene=3*.01*nCells # 3 counts in 1% of cells
print("minCountsPerGene: ", minCountsPerGene)

minSamples=.01*nCells # 1% of cells
print("minSamples: ", minSamples)

minCountsPerGene:  423.09
minSamples:  141.03


In [8]:
adata_copy = adata.copy()

# initial cuts
sc.pp.filter_cells(adata_copy, min_genes=200 )
sc.pp.filter_genes(adata_copy, min_cells=minSamples)
sc.pp.filter_genes(adata_copy, min_counts=minCountsPerGene)

adata_copy

AnnData object with n_obs × n_vars = 14038 × 8095
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent_mt', 'batch', 'label', 'new_label', 'condition', 'n_genes'
    var: 'n_cells', 'n_counts'

In [12]:
f_loom_path_scenic = "Data/nk_mm_scenic.loom"

# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
}
lp.create( f_loom_path_scenic, adata.X.transpose(), row_attrs, col_attrs)



## Scenic


### STEP 1: Gene regulatory network inference, and generation of co-expression modules
Phase Ia: GRN inference using the GRNBoost2 algorithm

For this step the CLI version of SCENIC is used.


In [14]:
tfs = "databases/allTFs_hg38.txt"

In [ ]:
!pyscenic grn {f_loom_path_scenic} {tfs} -o Results/adj.csv --num_workers 8 --seed 123


2023-02-15 14:45:32,875 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2023-02-15 14:45:36,795 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
preparing dask client
parsing input
creating dask graph
8 partitions
computing dask graph


In [ ]:
adjacencies = pd.read_csv("adj.tsv", index_col=False, sep='\t')
adjacencies.head()